<a href="https://colab.research.google.com/github/xpertdesh/ml-class21/blob/main/notes/bach_dummies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip

In [ ]:
! unzip bach.zip -d bach_chorales

##1. Import Neccessary Libraries
<br><br><br>

In [5]:
#data analysis libraries 
import numpy as np
import pandas as pd

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
bach = pd.read_csv('bach_chorales/bach.csv')
bach.describe(include='all')

#5. Cleaning Data

Time to clean our data, hot-encode the bass and notes columns, and delete the unnecessary columns.
<br><br><br>

<br><br>
* We need to drop `choral_ID`, and `event_number`.
* We need to hot-encode all the 12 notes.
* We need to hot-encode the `bass` column as well.
<br><br><br>

#### Choral Id and Event Number

In [13]:
# We'll start off by dropping the choral_id and event_number columns,
# since not a lot of useful information can be extracted from it
bach = bach.drop(['choral_ID'], axis=1)
bach = bach.drop(['event_number'], axis=1)

## One-hot encoding all the data
<br><br><br>
Using dummies

In [66]:
#create a new Dataframe to test out different methods
attempt1 = bach

# one-hot encode the notes of a Western Scale
attempt1 = attempt1.replace( to_replace = {"NO" : 0, "YES" : 1})

# one-hot encode the bass column
ohe = pd.get_dummies(attempt1['bass'], prefix="bass")
attempt1 = attempt1.drop('bass', axis=1)
attempt1 = attempt1.join(ohe)

# one-hot encode the meter column
ohe2 = pd.get_dummies(attempt1['meter'], prefix="level")
attempt1 = attempt1.drop('meter', axis=1)
attempt1 = attempt1.join(ohe2)

In [67]:
# split the features and labels columns
bFeatures = attempt1.drop('chord_label', axis=1)
bLabels = attempt1['chord_label']

#split the features/labels into a test and train dataframe
from sklearn.model_selection import train_test_split
bach_train_features, bach_test_features, bach_train_labels, bach_test_labels = train_test_split(bFeatures, bLabels, test_size = 0.2, random_state=42)

<br><br><br>
## Testing different models
<br><br><br>

In [71]:
# kNN Euclidean
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(bach_train_features, bach_train_labels)
predictions = knn.predict(bach_test_features)

from sklearn.metrics import accuracy_score
accuracy_score(bach_test_labels, predictions)

0.6902030008826125

In [69]:
# Knn Manhattan
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3, p=1)
knn.fit(bach_train_features, bach_train_labels)
predictions = knn.predict(bach_test_features)

from sklearn.metrics import accuracy_score
accuracy_score(bach_test_labels, predictions)

0.6902030008826125

In [72]:
# Decision Tree
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy')
clf.fit(bach_train_features, bach_train_labels)
predictions = clf.predict(bach_test_features)

from sklearn.metrics import accuracy_score
accuracy_score(bach_test_labels, predictions)

0.7157987643424537

In [73]:
# Decision Tree with cross validation
from sklearn.model_selection import GridSearchCV
hyperparam_grid = [
    {'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
     'min_samples_split': [2,3,4, 5]}
  ]

from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy')
grid_search = GridSearchCV(clf, hyperparam_grid, cv=10)
grid_search.fit(bach_train_features, bach_train_labels)

from sklearn.metrics import accuracy_score
predictions = grid_search.best_estimator_.predict(bach_test_features)
accuracy_score(bach_test_labels, predictions)

0.7096204766107679